In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

from art.defences.trainer import AdversarialTrainerAWPPyTorch

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            # 'TPR': '',
            # 'FNR': '',
            # 'TNR': '',
            # 'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/Defense/AWP/AWP_BIM.csv", mode='a', index=False)

In [3]:
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, f1_score,recall_score, roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def calculate_performance_metrics(x_test_adv, y_test, classifier, model_name, attack_name, epsilon):

    start_time = time.time()

    preds = np.argmax(classifier.predict(x_test_adv), axis=-1)
    # preds = classifier.predict(x_test_adv)

    y_hat = preds
    # y_test_arg = np.argmax(y_test, axis=1)
    y_test_arg = y_test


    accuracy = accuracy_score(y_test_arg, y_hat)
    precision_macro = precision_score(y_test_arg, y_hat, average='macro')
    # precision_micro = precision_score(y_test_arg, y_hat, average='micro')
    precision_weighted = precision_score(y_test_arg, y_hat, average='weighted')
    f1_macro = f1_score(y_test_arg, y_hat, average='macro')
    # f1_micro = f1_score(y_test_arg, y_hat, average='micro')
    f1_weighted = f1_score(y_test_arg, y_hat, average='weighted')
    recall_macro = recall_score(y_test_arg, y_hat, average='macro')
    # recall_micro = recall_score(y_test_arg, y_hat, average='micro')
    recall_weighted = recall_score(y_test_arg, y_hat, average='weighted')


    print(f"epsilon = {epsilon}")

    print(f"Accuracy: {accuracy}")
    # print(f"Precision(micro,macro,weighted): {precision_micro},{precision_macro},{precision_weighted}")
    # print(f"F1(micro,macro,weighted: {f1_micro},{f1_macro},{f1_weighted}")
    # print(f"Recall(micro,macro,weighted: {recall_micro},{recall_macro},{recall_weighted}")
    print(f"Precision(macro,weighted): {precision_macro},{precision_weighted}")
    print(f"F1(macro,weighted: {f1_macro},{f1_weighted}")
    print(f"Recall(macro,weighted: {recall_macro},{recall_weighted}")


    
    metrics = {
        "model" : model_name,
        "attack_model" : attack_name,
        "epsilon" : epsilon,
        "Accuracy": accuracy,
        # "Precision-micro": precision_micro,
        "Precision-macro": precision_macro,
        "Precision-weighted": precision_weighted,
        # "Recall_micro": recall_micro,
        "Recall_macro": recall_macro,
        "Recall_weighted": recall_weighted,
        
        # "F1-micro": f1_micro,
        "F1-macro": f1_macro,
        "F1-weighted": f1_weighted,

    }
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv("/home/jovyan/Defense/AWP/AWP_BIM.csv", mode='a', index=False, header=False)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time: {elapsed_time:.2f} second\n\n")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
x_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
x_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
x_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')


# train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

# val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
# val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [7]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [8]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Wustl_iiot/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [9]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_val = x_val.astype(np.float32)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [11]:
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent

FGSM = FastGradientMethod(estimator=classifier, eps=0.3)
BIM = BasicIterativeMethod(estimator=classifier, eps=0.3, max_iter=3, verbose = False)
PGD = ProjectedGradientDescent(estimator=classifier, eps=0.3, max_iter=3)

In [12]:
FGSM_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, FGSM, "TRADES", 0.1, 6.0, 0)
BIM_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, BIM, "TRADES", 0.1, 6.0, 0)
PGD_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, PGD, "TRADES", 0.1, 6.0, 0)

In [13]:
# start_time = time.time()
# FGSM_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [15]:
start_time = time.time()
BIM_Trainer.fit(x_train, y_train, epochs=5, batch_size=128,
                     validation_data=(x_val, y_val))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time: {elapsed_time:.2f} second\n\n")

Adversarial Training AWP with TRADES - Epochs: 100%|██████████| 20/20 [4:19:22<00:00, 778.11s/it]  

Time: 15562.24 second




In [15]:
# start_time = time.time()
# PGD_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [26]:
calculate_performance_metrics(x_test, y_test, BIM_Trainer, 'BIM_Trainer', 'baseline', '0')
# calculate_performance_metrics(x_test, y_test, FGSM_Trainer, 'FGSM_Trainer', 'baseline', '0')
# calculate_performance_metrics(x_test, y_test, PGD_Trainer, 'PGD_Trainer', 'baseline', '0')

epsilon = 0
Accuracy: 0.9998659702705306
Precision(macro,weighted): 0.8992601763938672,0.9998892769560725
F1(macro,weighted: 0.9139629700221977,0.9998735977989378
Recall(macro,weighted: 0.9357276218926704,0.9998659702705306
Time: 1.42 second




In [25]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'BIM', epsilon)

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'FGSM', epsilon)

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, PGD_Trainer, 'PGD_Trainer', 'PGD', epsilon)

epsilon = 0.01
Accuracy: 0.999493200085444
Precision(macro,weighted): 0.836202806671583,0.9997979760363629
F1(macro,weighted: 0.8665067685003798,0.9996060525381799
Recall(macro,weighted: 0.9668619404728769,0.999493200085444
Time: 1.46 second


epsilon = 0.1
Accuracy: 0.006098352690856241
Precision(macro,weighted): 0.03920746675010343,0.07052984024263906
F1(macro,weighted: 0.022585288848983576,0.009607458792202497
Recall(macro,weighted: 0.10454538102050237,0.006098352690856241
Time: 1.41 second


epsilon = 0.2
Accuracy: 0.0027727400283975487
Precision(macro,weighted): 0.00836860422805645,0.036543342814229475
F1(macro,weighted: 0.0012088674331609396,0.004786073547271727
Recall(macro,weighted: 0.0067888553714719165,0.0027727400283975487
Time: 1.41 second


epsilon = 0.3
Accuracy: 0.002391592985219034
Precision(macro,weighted): 0.007374402686297301,0.0341843669945908
F1(macro,weighted: 0.0009643804151395904,0.004470427699152436
Recall(macro,weighted: 0.0005159250056471651,0.002391592985219

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
torch.save(BIM_Trainer, "/home/jovyan/Defense/AWP/AWP_BIM.pt")

In [ ]:
# skip start here

In [13]:
BIM_Trainer = torch.load("/home/jovyan/Defense/AWP/AWP_BIM.pt")


In [17]:
calculate_performance_metrics(x_test, y_test, BIM_Trainer, 'BIM_Trainer', 'baseline', '0')

epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
print("start BIM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'BIM', epsilon)

print("start FGSM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'FGSM', epsilon)

print("start PGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'PGD', epsilon)

epsilon_values = [0.01, 0.1, 0.2, 0.3]

print("start DF")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'DF', epsilon)
    
# Iterate over epsilon values
print("start AutoPGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'AutoPGD', epsilon)

print("start BA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'BA', epsilon)

print("start HopSkipJump")
filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_HopSkipJump_eps_0.npy'
x_test_adv = np.load(filename)

calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'HopSkipJump', '0')

print("start ZOO")
ZOO_epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]
# Iterate over epsilon values
for epsilon in ZOO_epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'ZOO', epsilon)

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0
Accuracy: 0.9979769887708217
Precision(macro,weighted): 0.7903507269750408,0.9978215392902711
F1(macro,weighted: 0.7243565987227868,0.9977659358010255
Recall(macro,weighted: 0.6794342793129201,0.9979769887708217
Time: 3.28 second


start BIM


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.01
Accuracy: 0.9974157392786688
Precision(macro,weighted): 0.7886169876868939,0.9972830664448457
F1(macro,weighted: 0.7124472072373625,0.9971078227269026
Recall(macro,weighted: 0.6631721433905899,0.9974157392786688
Time: 3.29 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.1
Accuracy: 0.9979769887708217
Precision(macro,weighted): 0.790072910123644,0.9978184131196459
F1(macro,weighted: 0.7242581515759381,0.9977659217468832
Recall(macro,weighted: 0.6794342793129201,0.9979769887708217
Time: 3.29 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.2
Accuracy: 0.9979769887708217
Precision(macro,weighted): 0.790072910123644,0.9978184131196459
F1(macro,weighted: 0.7242581515759381,0.9977659217468832
Recall(macro,weighted: 0.6794342793129201,0.9979769887708217
Time: 3.29 second


epsilon = 0.3
Accuracy: 0.9979769887708217
Precision(macro,weighted): 0.790072910123644,0.9978184131196459
F1(macro,weighted: 0.7242581515759381,0.9977659217468832
Recall(macro,weighted: 0.6794342793129201,0.9979769887708217
Time: 3.29 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
